In [2]:
#To run in Jupyter Notebook:
%run reading_3sat 
%run read_histogram
%run graph_to_3sat

#Imports
from reading_3sat import *
from qiskit import QuantumRegister, ClassicalRegister
import math
from qiskit.circuit.library import MCMT
from qiskit.circuit.library.standard_gates import *
from qiskit import QuantumCircuit, execute, Aer
from qiskit.tools.monitor import job_monitor
import numpy as np
from qiskit.visualization import plot_histogram
from qiskit.tools.visualization import plot_bloch_multivector
from qiskit.circuit.library import QFT
import time

backend = Aer.get_backend('qasm_simulator')
pi = np.pi

In [3]:
def OR_gates(circuit, clauses):
    for clause in clauses:
        input_qubits = []  #Create a list of qubits for the mct gate 
        clause_num = clauses.index(clause) #Save the index of this clause as "clause_num"  
                                            #(=the qubit used for this OR gate result)
        
        for c in clause:
            #Add a NOT gate for each variable with no ¬ and add the variable in 'input_qubits'
            if "¬" not in c:
                variable_num = variables.index(c) #This is the index of the qubit that represents this variable
                                                        
                input_qubits.append(variable_num)
                circuit.x(variable_num)
            else:
                variable_num = variables.index(c[1:]) #The index is used to not include the negation symbol
                input_qubits.append(variable_num)
        
        #Add a NOT gate before applying a multi-controlled toffoli gate, to simulate an OR gate
        circuit.x(len(variables) + clause_num)
        circuit.mct(input_qubits, len(variables) + clause_num)

        for c in clause:
            #Add another NOT for each variable with no ¬
            if "¬" not in c:
                variable_num = variables.index(c) 
                circuit.x(variable_num)   
    return

def oracle(n, print_flag = 0):
    qc = QuantumCircuit(n)
    
    #Create the OR gates
    OR_gates(qc, clauses)

    #Multiply the OR-gate(s) results; control qubit becomes 1 when all of them are 1
    and_inputs = []
    for i in range(len(variables)-ancilla_no, len(variables) + len(clauses)):
        and_inputs.append(i)
        
    if len(clauses):
        qc.mct(and_inputs, -1)
    else:
        qc.x(-1)

    #Mirroring
    OR_gates(qc, clauses)
    if print_flag==1:
        print(qc)
    return qc.to_gate(label=" Oracle ")

def diffuser(n, print_flag = 0):
    qc = QuantumCircuit(n)
    
    #Create a list of the qubits and initialize 
    var_qubits = []
    for i in range(n):
        var_qubits.append(i)
        qc.h(i)
        qc.x(i)
    
    #Simulate a multicontrolled Z-gate
    qc.h(var_qubits[-1])
    qc.mct(var_qubits[0:-1], var_qubits[-1])
    qc.h(var_qubits[-1])
    
    #NOT and HADAMARD gates to all qubits
    for i in range(n):
        qc.x(i)
        qc.h(i)
    if print_flag==1:
        print(qc)
    return qc.to_gate(label=" Diffuser ")

#-------------------------------------------------------------------------------------------------------------------------------

#CNF INPUT
graph = {
    'A': ['E'],
    'B': ['D'],
    'C': ['E'],
    'D': ['E', 'B'],
    'E': ['A', 'C', 'D']
}
colors_no = 2
cnf = graph_to_3sat(graph, colors_no)
print("3-SAT expression:", cnf)
all_variables = save_variables(cnf)
cnf_array = []
print("Number of variables:", len(all_variables)) 
upper_limit = 20
clauses_per_section = upper_limit - len(all_variables) - 6
if (clauses_per_section < 1):
    clauses_per_section = 1
print("Maximum clauses to be implemented per time:", clauses_per_section) 

sections_num = math.floor(len(cnf)/clauses_per_section) 

#Split in equal sections (up to max_clauses per time)
for i in range(sections_num):
    cnf1 = cnf[i*clauses_per_section:(i+1)*clauses_per_section]
    cnf_array.append(cnf1)

#Last part (if there are clauses left)
if len(cnf)%clauses_per_section:
    cnf1 = cnf[len(cnf)-len(cnf)%clauses_per_section:]
    cnf_array.append(cnf1)

t0 = time.time()

solutions = []
for section in cnf_array:
    print("\n\n---Part %d out of %d---" %(cnf_array.index(section) + 1, len(cnf_array)))
    
    #Create arrays for variables and clauses
    variables = save_variables(section)
    clauses = simplify_clauses(section)      

    print("Clauses to be implemented:", section)
    N = len(variables)
    C = len(clauses)    
    ancilla_no = 0

    #----GROVER GATE CREATION----

    #Initialization
    q = QuantumRegister(N + C + 1,'q')     
    c = ClassicalRegister(N,'c')
    grover_gate = QuantumCircuit(q)

    #Oracle
    grover_gate.append(oracle(N + C + 1), range(N + C + 1))

    #Diffuser
    grover_gate.append(diffuser(N), range(N))   

    #Add control
    grover = grover_gate.to_gate(label = ' G ')


    # ----QUANTUM PHASE ESTIMATION--------------------------------------------------------------------------------------------------
    t = 6
    qpe = QuantumCircuit(t + N + C + 1, t)    

    # Prepare initial state
    for i in range(t + N):
        qpe.h(i)

    #Control qubit gets initialized
    qpe.x(t + N + C)
    qpe.h(t + N + C)

    qpe.barrier()

    # Controlled-U operations:
    gate = grover.control()
    gate.calibrations = qpe.calibrations
    repetitions = 1
    target = []
    for i in range(t, t + N + C + 1):
        target.append(i)
    for q in range(t):
        for i in range(repetitions):
            qpe.append(gate, [q] + target);
        repetitions *= 2

    # Inverse QFT:
    qpe = qpe.compose(QFT(t, inverse=True), range(t))

    # Measurements
    qpe.barrier()
    qpe.measure(range(t),range(t)) 

    #Run circuit
    #backend1 = Aer.get_backend('qasm_simulator')
    counts = execute(qpe, backend, shots=1024).result().get_counts()

    #Estimate phase and number of solutions (m) 
    reading = max(counts, key = lambda x: counts[x])
    print("QPE reading:", reading)
    phase = (int(reading, 2)/2**t)
    theta = 2*pi*phase
    m = (2**N)*((math.sin(theta/2))**2)
    m = (2**N) - round(m)

    print("Estimated phase:", phase)
    print("Estimated solutions: %i out of %i" %(m, (2**N)))

    iterations = 1
    if m == 0:
        print("This 3_SAT input is estimated not to be satisfiable.")
    elif m == 2**N:
        print("This 3_SAT input is estimated to always be satisfiable.")
    elif m<2**N/2:
        iterations = math.floor((pi/4)*((2**N)/m)**0.5)
        print("Estimated iterations:", iterations)

    #DOUBLING THE SEARCH SPACE
    #Adding ancilla qubits when m is more than half the search space
    else:
        if m>=2**N/2 and m<0.75*2**N:
            print("Adding an ancilla qubit")
            ancilla_no = 1
            for i in range(ancilla_no):
                variables.append('a' + str(i))
            
        if m>=0.75*2**N:
            print("Adding 2 ancilla qubits")
            ancilla_no = 2
            for i in range(ancilla_no):
                variables.append('a' + str(i))
        N = len(variables)
        C = len(clauses)    

        #NEW GROVER OPERATOR
        #Initialization
        q = QuantumRegister(N + C + 1,'q')     
        c = ClassicalRegister(N,'c')
        grover_gate = QuantumCircuit(q)

        #Oracle
        grover_gate.append(oracle(N + C + 1), range(N + C + 1))

        #Diffuser
        grover_gate.append(diffuser(N), range(N))   

        #Add control
        grover = grover_gate.to_gate(label = ' G ')

        #New iterations
        iterations = math.floor((pi/4)*((2**N)/m)**0.5)
        print("Estimated iterations:", iterations)
            

    #-------------------------------------------------------------------------------------------------------------------------------
    #----GROVER----    

    #INITIALIZATION 
    q = QuantumRegister(N + C + 1,'q')     
    c = ClassicalRegister(N - ancilla_no,'c')
    circuit = QuantumCircuit(q,c)

    #All input qubits get initialized with a Hadamard gate                         
    i=0
    while (i<N):
        circuit.h(q[i])
        i+=1

    #Control qubit gets initialized
    circuit.x(q[-1])
    circuit.h(q[-1])

    circuit.barrier()

    #ITERATIONS
    #After the initialization circuit, the main circuit is repeated as many times as needed
    print("Oracle and grover operator will be repeated", iterations, "time(s)")

    #Adding one combination of O-G for each iteration
    for i in range(iterations):
        circuit.append(grover, range(N + C + 1))

    #MEASUREMENTS
    for i in range(0, N - ancilla_no):
        circuit.measure(q[i], N - ancilla_no - 1 - i)
        
    #RUN CIRCUIT
    job = execute(circuit, backend, shots=2048)
    job_monitor(job)
    counts = job.result().get_counts()
    
    result = read_data(counts, clauses, variables[:N - ancilla_no])
    if not len(result):
        print("There are no solutions for this graph")
        break
        
    #EDIT THE SOLUTIONS
    #Each section could involve less variables that the total used in the 3-SAT problem. 
    #Therefore, we should pad each solution with '0' and '1' to match the length of the total variables 
    current_solutions = {}
    for sol in result:
        a = ['']
        for var in all_variables:
            if var in variables:
                for j in range(len(a)):
                    a[j] = a[j] + sol[variables.index(var)]
            else:
                length_a = len(a)
                for j in range(length_a):
                    a.append(a[j] + '1')
                    a[j] = a[j] + '0'
        for s in a:    
            current_solutions[s] = 1
    solutions.append(current_solutions)  
    
    
#Keep only the common solutions (the ones that satisfy all sections of the 3-SAT problem)
final_result = []

for sol in min(solutions, key=len):
    counter = 0
    for compare_array in solutions:
        if sol in compare_array:
            counter = counter + 1
    if counter == len(solutions) and sol not in final_result:
        final_result.append(sol)

#Printing the final result
print("\n\n---Final Result---\n")
if len(final_result):
    print(all_variables, "=", final_result)
else:
    print("There are no solutions for this graph")
print ("Time:", time.time() - t0, "seconds")

3-SAT of 26 clauses
3-SAT expression: [['A_1', 'A_2', 'A_1'], ['B_1', 'B_2', 'B_1'], ['C_1', 'C_2', 'C_1'], ['D_1', 'D_2', 'D_1'], ['E_1', 'E_2', 'E_1'], ['¬A_1', '¬A_2', '¬A_1'], ['¬B_1', '¬B_2', '¬B_1'], ['¬C_1', '¬C_2', '¬C_1'], ['¬D_1', '¬D_2', '¬D_1'], ['¬E_1', '¬E_2', '¬E_1'], ['¬A_1', '¬E_1', '¬A_1'], ['¬A_2', '¬E_2', '¬A_2'], ['¬B_1', '¬D_1', '¬B_1'], ['¬B_2', '¬D_2', '¬B_2'], ['¬C_1', '¬E_1', '¬C_1'], ['¬C_2', '¬E_2', '¬C_2'], ['¬D_1', '¬E_1', '¬D_1'], ['¬D_2', '¬E_2', '¬D_2'], ['¬D_1', '¬B_1', '¬D_1'], ['¬D_2', '¬B_2', '¬D_2'], ['¬E_1', '¬A_1', '¬E_1'], ['¬E_2', '¬A_2', '¬E_2'], ['¬E_1', '¬C_1', '¬E_1'], ['¬E_2', '¬C_2', '¬E_2'], ['¬E_1', '¬D_1', '¬E_1'], ['¬E_2', '¬D_2', '¬E_2']]
Number of variables: 10
Maximum clauses to be implemented per time: 4


---Part 1 out of 7---
Clauses to be implemented: [['A_2', 'A_1'], ['B_2', 'B_1'], ['C_2', 'C_1'], ['D_2', 'D_1']]
QPE reading: 010100
Estimated phase: 0.3125
Estimated solutions: 79 out of 256
Estimated iterations: 1
Oracle and 

In [5]:
def result_format(final_result, all_variables, colors):
    '''
    Translates the final result array to a user friendly response.
    Example use:
    Input: 
        final_result(list): ['100010001100']
        all_variables(list): ['A_1', 'A_2', 'A_3', 'B_1', 'B_2', 'B_3', 'C_1', 'C_2', 'C_3', 'D_1', 'D_2', 'D_3']
        colors(list): ["Red", "Green", "Blue"]
    Printed output:
        Solution 1:
        Node A = Red
        Node B = Green
        Node C = Blue
        Node D = Red              
    '''
    for solution in final_result:
        print("\nSolution %d:" %(final_result.index(solution)+1))
        for i in range(len(all_variables)):
            if int(solution[i]):
                print("Node %s = %s" %(all_variables[i][0], colors[int(all_variables[i][2])-1]))
            
colors = ["Red", "Green", "Blue", "Yellow", "Orange", "Pink", "Purple", "Brown"]
result_format(final_result, all_variables, colors[:colors_no])


Solution 1:
Node A = Red
Node B = Green
Node C = Red
Node D = Red
Node E = Green

Solution 2:
Node A = Green
Node B = Red
Node C = Green
Node D = Green
Node E = Red


In [4]:
#Example to display functionality of padding each section's result
variables = [x, y, z]
all_variables = [x, y, z, a]
result = ["000", "111"]
#EDIT THE SOLUTIONS
#Each section could involve less variables that the total used in the 3-SAT problem. 
#Therefore, we should pad each solution with '0' and '1' to match the length of the total variables 
current_solutions = []
for sol in result:
    a = ['']
    for var in all_variables:
        if var in variables:
            for j in range(len(a)):
                a[j] = a[j] + sol[variables.index(var)]
        else:
            length_a = len(a)
            for j in range(length_a):
                a.append(a[j] + '1')
                a[j] = a[j] + '0'
    for s in a:    
        current_solutions.append(s)
print(current_solutions)

['0000', '0001', '1110', '1111']
